In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("Book-Summary-Rich-Dad.pdf") 
docs = loader.load()

In [2]:
docs

[Document(page_content="Rich Dad Poor Dad was first published in 1997, and has racked up sales of 32 million copies in more than        \n51 languages. Endorsed by many celebrities including Oprah Winfrey, this book stayed on the New York Times \nbest-sellers list for six years. This handcrafted summary of Rich Dad Poor Dad summarises the six rules that \nthe authors believe will help your money work for you, rather than the other way around. The summary \ncaptures a few of the fables that the authors recount as part of explaining the six rules.\n Poor Dad versus Rich Dad\n'The love of money is the root of all' versus  'The lack of money is the root of all evil’\n'I cannot afford it' versus 'How can I afford it?'\n'Money does not matter' versus 'Money is power’\n'Study hard so you can find a good company to work for' versus 'Study hard so you can find a good company to \nbuy’\nOne was my Poor Dad while the other was my Rich Dad.\nHow did I end up with two dads? One, obviously, was my b

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000 , chunk_overlap=200)
doc =text_splitter.split_documents(docs)


In [4]:
#vector Embedding and Vector store
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents=doc, embedding=OllamaEmbeddings(model="llama3",show_progress=True))
retriver = db.as_retriever(search_type="mmr",search_kwargs={'k': 6, 'lambda_mult': 0.25})

OllamaEmbeddings: 100%|██████████| 21/21 [04:41<00:00, 13.42s/it]


In [10]:
#vector database
query = "Abstract"
result = db.similarity_search(query)
result[0].page_content

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.61s/it]


'The value of the program counter\n(PC) indicates the address of the next instruction\nof the process being executed\n.\nEach process is represented by a Process Control Block\n(PCB).\nApni Kaksha\n1'

In [12]:
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
import streamlit as st

prmpt = PromptTemplate.from_template("""You are an assistant for question-answering tasks. Use the following pieces of
                                         retrieved context to answer the question. If you don't know the answer, just say that 
                                         you don't know. Use three sentences maximum and keep the answer concise.
Question : {question}
Context : {context}""")

llm = Ollama(model="llama3")

OutputParser = StrOutputParser()

rag_chain = (
    {"context": retriver , "question" :RunnablePassthrough()}
    |prmpt
    |llm
    |OutputParser
)

rag_chain.invoke("Batch OS")


OllamaEmbeddings:   0%|          | 0/1 [00:00<?, ?it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.09s/it]


"I don't know about batch OS. The context doesn't mention anything specific to batch operating systems."